In [2]:
from trainer import CatDogTrainer
import tensorflow as tf

%load_ext autoreload
%autoreload 2

It's very important to notice that even in the original case validation accuracy is **higher** than training (and loss is lower).

# 1 - original training

Here we train the original model from [here](https://www.tensorflow.org/tutorials/images/transfer_learning). We get very good accuracy on both train and validation sets (more than .9) after just 2-3 epochs.

In [3]:
trainer = CatDogTrainer(initial_epochs=3, model_type='mobile_net')

In [4]:
trainer.train()

Epoch 1/3
582/582 [==============================] - 77s 132ms/step - loss: 2.5899 - accuracy: 0.6847 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/3
582/582 [==============================] - 39s 66ms/step - loss: 1.4116 - accuracy: 0.8271 - val_loss: 0.9865 - val_accuracy: 0.8871
Epoch 3/3
582/582 [==============================] - 39s 67ms/step - loss: 0.9741 - accuracy: 0.8763 - val_loss: 0.7353 - val_accuracy: 0.9181


# 2 - resnet

Let's now change `MobileNetV2` to `resnet50` without changing anything else. Result are not bad for validation set but for some reason train accuracy is below than validation accuracy.

In [5]:
trainer = CatDogTrainer(initial_epochs=3, model_type='resnet50')

94674944/94668760 [==============================] - 1s 0us/step


In [6]:
trainer.train()

Epoch 1/3
582/582 [==============================] - 59s 102ms/step - loss: 2.4497 - accuracy: 0.6938 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/3
582/582 [==============================] - 53s 91ms/step - loss: 1.3165 - accuracy: 0.8233 - val_loss: 0.6755 - val_accuracy: 0.9427
Epoch 3/3
582/582 [==============================] - 53s 91ms/step - loss: 1.0544 - accuracy: 0.8525 - val_loss: 0.5614 - val_accuracy: 0.9509


# 3 - changing `mobile_net` version

## image resolution

Let's first change image resolution - from 160 to 224. So everything works and the result is even better.

In [7]:
trainer = CatDogTrainer(initial_epochs=3, model_type='mobile_net')

In [8]:
trainer.train()

Epoch 1/3
582/582 [==============================] - 59s 101ms/step - loss: 1.8345 - accuracy: 0.7266 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/3
582/582 [==============================] - 53s 91ms/step - loss: 0.7916 - accuracy: 0.8763 - val_loss: 0.3933 - val_accuracy: 0.9422
Epoch 3/3
582/582 [==============================] - 53s 91ms/step - loss: 0.5294 - accuracy: 0.9140 - val_loss: 0.3870 - val_accuracy: 0.9435


## normalization

Now let's remove strange normalization and 1 class:

```python
def format_example(image, label):
    image = tf.cast(image, tf.float32)
    # let's change this to image /= 255.0
    image = (image / 127.5) - 1
    image = tf.image.resize(image, (self.IMG_SIZE, self.IMG_SIZE))
    return image, label

# change this to 2
self.prediction_layer = tf.keras.layers.Dense(1)

```

In [10]:
trainer = CatDogTrainer(initial_epochs=3, model_type='mobile_net')

First let's check our changes. We have 2 classes and numbers are in range `[0, 1]`.

In [11]:
trainer.model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_3 ( (None, 1280)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 2562      
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________


In [12]:
img_batch, label_batch = next(iter(trainer.train_batches))

In [13]:
img_batch.shape

TensorShape([32, 224, 224, 3])

In [14]:
img_batch[0, 0, 0, :]

<tf.Tensor: id=76276, shape=(3,), dtype=float32, numpy=array([0.68132335, 0.6166525 , 0.5587918 ], dtype=float32)>

Let's now train the model. Look's like everything still works.

In [15]:
trainer.train()

Epoch 1/3
582/582 [==============================] - 55s 95ms/step - loss: 2.2397 - accuracy: 0.6922 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/3
582/582 [==============================] - 49s 85ms/step - loss: 0.8954 - accuracy: 0.8733 - val_loss: 0.8265 - val_accuracy: 0.8974
Epoch 3/3
582/582 [==============================] - 50s 87ms/step - loss: 0.5853 - accuracy: 0.9134 - val_loss: 0.7149 - val_accuracy: 0.9138


## loss and metrics

It's not clear what all these means (`accuracy` etc.). Let's first look at our labels. They are **not** one-hot.

In [16]:
label_batch.shape

TensorShape([32])

In [17]:
label_batch[:5]

<tf.Tensor: id=87757, shape=(5,), dtype=int64, numpy=array([1, 0, 0, 0, 1])>

Can we use `SparseCategoricalCrossentropy`? It's not quite clear - result are mixed at best. Why is that? Results should be the same. Let's also try to use `BinaryCrossentropy` that is perfect fit in our case. It works.

In [18]:
trainer = CatDogTrainer(initial_epochs=3, model_type='mobile_net')

In [19]:
trainer.loss

In [20]:
trainer.train()

Epoch 1/3
582/582 [==============================] - 55s 95ms/step - loss: 0.8347 - accuracy: 0.6128 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/3
582/582 [==============================] - 51s 87ms/step - loss: 0.6811 - accuracy: 0.5992 - val_loss: 0.6788 - val_accuracy: 0.7983
Epoch 3/3
582/582 [==============================] - 50s 86ms/step - loss: 0.6747 - accuracy: 0.5911 - val_loss: 0.6687 - val_accuracy: 0.7422


In [21]:
trainer = CatDogTrainer(initial_epochs=3, model_type='mobile_net')

In [22]:
trainer.loss

In [23]:
trainer.train()

Epoch 1/3
582/582 [==============================] - 54s 93ms/step - loss: 2.2462 - accuracy: 0.6996 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/3
582/582 [==============================] - 50s 86ms/step - loss: 0.9856 - accuracy: 0.8555 - val_loss: 0.9722 - val_accuracy: 0.8707
Epoch 3/3
582/582 [==============================] - 50s 85ms/step - loss: 0.6604 - accuracy: 0.9024 - val_loss: 0.9002 - val_accuracy: 0.8860


Let's try to set seed. Looks like it works!

In [26]:
tf.random.set_seed(42)

In [27]:
trainer = CatDogTrainer(initial_epochs=1, model_type='mobile_net')

In [28]:
trainer.train()

582/582 [==============================] - 54s 92ms/step - loss: 1.8132 - accuracy: 0.7372 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00


In [29]:
tf.random.set_seed(42)
trainer = CatDogTrainer(initial_epochs=1, model_type='mobile_net')
trainer.train()

582/582 [==============================] - 54s 92ms/step - loss: 1.8132 - accuracy: 0.7372 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00


Let's now change `accuracy` to `tf.keras.metrics.Accuracy`. It doesn't work! So we need to be very careful with `tf.keras` API.

In [39]:
tf.random.set_seed(42)
trainer = CatDogTrainer(initial_epochs=1, model_type='mobile_net')

In [41]:
trainer.metrics

In [42]:
trainer.train()

582/582 [==============================] - 55s 94ms/step - loss: 1.8132 - binary_accuracy: 0.7372 - val_loss: 0.0000e+00 - val_binary_accuracy: 0.0000e+00


In [14]:
tf.random.set_seed(42)
trainer = CatDogTrainer(initial_epochs=1, model_type='mobile_net')

In [15]:
trainer.metrics

['accuracy']

In [16]:
trainer.train()

582/582 [==============================] - 56s 96ms/step - loss: 1.7554 - accuracy: 0.7462 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00


In [3]:
tf.random.set_seed(42)
trainer = CatDogTrainer(initial_epochs=1, model_type='mobile_net')

In [4]:
trainer.metrics

In [5]:
trainer.train()

      1/Unknown - 1s 1s/step

ValueError: Shapes (None, 2) and (None, 1) are incompatible

## optimizer and learning rates

# 4 - `resnet50`

Let's now try to change the model to `resnet50`. So looks like everything works even with the change of the net. But what if we change our optimizer?

In [9]:
tf.random.set_seed(42)
trainer = CatDogTrainer(initial_epochs=3, model_type='resnet50')

In [8]:
trainer.base_model.name

'resnet50v2'

In [10]:
trainer.train()

Epoch 1/3
582/582 [==============================] - 94s 161ms/step - loss: 1.7401 - binary_accuracy: 0.7644 - val_loss: 0.0000e+00 - val_binary_accuracy: 0.0000e+00ary_accuracy: 0.7 - 84s 147ms/step - loss: 1.7601 - binary_accuracy: - 84s 146ms/step - loss: 1.7456 - binary_accuracy: 0
Epoch 2/3
582/582 [==============================] - 84s 144ms/step - loss: 0.6661 - binary_accuracy: 0.9055 - val_loss: 0.3289 - val_binary_accuracy: 0.9659
Epoch 3/3
582/582 [==============================] - 83s 143ms/step - loss: 0.5235 - binary_accuracy: 0.9244 - val_loss: 0.2802 - val_binary_accuracy: 0.9726


In [11]:
tf.random.set_seed(42)
trainer = CatDogTrainer(initial_epochs=3, model_type='resnet50')

In [12]:
trainer.metrics

['accuracy']

In [13]:
trainer.train()

Epoch 1/3
582/582 [==============================] - 89s 152ms/step - loss: 1.7441 - accuracy: 0.7641 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/3
582/582 [==============================] - 86s 148ms/step - loss: 0.6715 - accuracy: 0.9052 - val_loss: 0.3092 - val_accuracy: 0.9677
Epoch 3/3
582/582 [==============================] - 85s 146ms/step - loss: 0.5171 - accuracy: 0.9250 - val_loss: 0.2855 - val_accuracy: 0.9722


In [17]:
tf.random.set_seed(42)
trainer = CatDogTrainer(initial_epochs=10, model_type='resnet50')

In [18]:
trainer.train()

Epoch 1/10
582/582 [==============================] - 89s 153ms/step - loss: 1.7482 - binary_accuracy: 0.7629 - val_loss: 0.0000e+00 - val_binary_accuracy: 0.0000e+00
Epoch 2/10
582/582 [==============================] - 85s 146ms/step - loss: 0.6828 - binary_accuracy: 0.9017 - val_loss: 0.3417 - val_binary_accuracy: 0.9647
Epoch 3/10
582/582 [==============================] - 84s 144ms/step - loss: 0.5244 - binary_accuracy: 0.9243 - val_loss: 0.2924 - val_binary_accuracy: 0.9713
Epoch 4/10
582/582 [==============================] - 84s 144ms/step - loss: 0.4544 - binary_accuracy: 0.9360 - val_loss: 0.2553 - val_binary_accuracy: 0.9759
Epoch 5/10
582/582 [==============================] - 85s 145ms/step - loss: 0.3966 - binary_accuracy: 0.9435 - val_loss: 0.2519 - val_binary_accuracy: 0.9776
Epoch 6/10
582/582 [==============================] - 83s 143ms/step - loss: 0.3809 - binary_accuracy: 0.9449 - val_loss: 0.2267 - val_binary_accuracy: 0.9774
Epoch 7/10
582/582 [==================